In [ ]:
import requests

response = requests.get("https://github.com/login/oauth/authorize", params={
    "client_id": "MY_CLIENT_ID",
    "redirect_uri": "http://localhost:3000/callback",
    "scope": "user",
    "state": "MY_STATE",
})
print(response.status_code)

In [ ]:
# Ver si se puede usar esto https://github.com/modelcontextprotocol/python-sdk/blob/main/src/mcp/client/auth.py

In [ ]:
# Cell 1: Setup
import requests
import webbrowser
import http.server
import socketserver
import urllib.parse

CLIENT_ID = ""
CLIENT_SECRET = ""
REDIRECT_URI = "http://localhost:8000/callback"
AUTH_URL = "https://github.com/login/oauth/authorize"
TOKEN_URL = "https://github.com/login/oauth/access_token"
USER_API_URL = "https://api.github.com/user"

received_code = None


In [ ]:
# Cell 2: Start OAuth flow

# Construct authorization URL
params = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "scope":"repo" # "read:user",
}
auth_request_url = f"{AUTH_URL}?{urllib.parse.urlencode(params)}"

# Open GitHub login page in browser
print("Opening browser for GitHub OAuth login...")
webbrowser.open(auth_request_url)


In [ ]:
# Cell 3: Create a basic HTTP server to receive the callback

class OAuthHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        global received_code
        parsed_path = urllib.parse.urlparse(self.path)
        query_params = urllib.parse.parse_qs(parsed_path.query)
        if parsed_path.path == "/callback" and "code" in query_params:
            received_code = query_params["code"][0]
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"<h1>You can close this tab now.</h1>")
        else:
            self.send_response(404)
            self.end_headers()

PORT = 8000
print("Waiting for GitHub callback...")
httpd = socketserver.TCPServer(("", PORT), OAuthHandler)
httpd.handle_request()  # serve one request and stop


In [ ]:
# Cell 4: Exchange code for access token

if not received_code:
    raise Exception("No code received.")

response = requests.post(
    TOKEN_URL,
    headers={"Accept": "application/json"},
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": received_code,
        "redirect_uri": REDIRECT_URI,
    },
)

token_data = response.json()
access_token = token_data.get("access_token")

print("Access Token:", access_token)


In [ ]:
# Acá se usa el access_token para logearse al mcp server

In [ ]:
# Cell 5: Use token to fetch GitHub user profile

if not access_token:
    raise Exception("No access token.")

user_response = requests.get(
    USER_API_URL,
    headers={"Authorization": f"token {access_token}"}
)

user_data = user_response.json()
print("GitHub User:", user_data["login"])
user_data
